First, let's write a simple naive function that transposes an ndarray. We'll start importing NumPy:

In [ ]:
import numpy as np

Then, writing such function:

In [ ]:
def transpose2d_naive(a: np.ndarray) -> np.ndarray:
    if a.ndim != 2:
        raise ValueError("transpose2d_naive expects a 2D array")
    m, n = a.shape
    out = np.empty((n, m), dtype=a.dtype)
    for i in range(m):
        for j in range(n):
            out[j, i] = a[i, j]
    return out

Next, we will install and import Numba.

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
pip install numba

In [ ]:
pip show numba

Now, we'll write a CUDA kernel using Numba. It will leverage the highly parallel architecture of our CUDA GPU.

In [ ]:
from numba import cuda, types as numba_types

@cuda.jit
def transpose2d_numba(a, transposed):
    # This function assumes it is launched with a 32x32 block dimension, and that `a` is a multiple of these dimensions.
    
    # Create 32x32 shared memory array.
    tile = cuda.shared.array((32, 33), numba_types.float32)

    # Compute offsets into global input array.
    x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # Make coalesced read from global memory into shared memory array.
    # Note the use of local thread indices for the shared memory write,
    # and global offsets for global memory read.
    tile[cuda.threadIdx.y, cuda.threadIdx.x] = a[y, x]

    # Wait for all threads in the block to finish updating shared memory.
    cuda.syncthreads()
    
    # Calculate transposed location for the shared memory array tile to be written back to global memory.
    t_x = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_y = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # Write back to global memory, transposing each element within the shared memory array.
    transposed[t_y, t_x] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

Next, we'll create the data we'll be operating with:

In [ ]:
n = 4096*4096 # 16M
threads_per_block = (32, 32)
blocks = (128, 128)

a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

Then, we'll check our naive transpose function (without CUDA/Numba):

In [ ]:
%timeit transposed = transpose2d_naive(a)

Therefore, we'll check our transpose function using CUDA/Numba:

In [ ]:
a = np.arange(n).reshape((4096,4096)).astype(np.float32)
transposed = np.zeros_like(a).astype(np.float32)

d_a = cuda.to_device(a)
d_transposed = cuda.to_device(transposed)

In [ ]:
%timeit transpose2d_numba[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()